In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from PIL import Image

In [47]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Print the shapes of the loaded data
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [40]:
for i in range(3):
    img = Image.fromarray(x_train[i], 'RGB')
    img.show()
x_test = x_test/255.0
x_train = x_train/255.0 #Normalized pixel values
print(x_train[0])
for i in range(3):
    img = Image.fromarray(x_train[i], 'RGB')
    img.show()


[[[0.23137255 0.24313725 0.24705882]
  [0.16862745 0.18039216 0.17647059]
  [0.19607843 0.18823529 0.16862745]
  ...
  [0.61960784 0.51764706 0.42352941]
  [0.59607843 0.49019608 0.4       ]
  [0.58039216 0.48627451 0.40392157]]

 [[0.0627451  0.07843137 0.07843137]
  [0.         0.         0.        ]
  [0.07058824 0.03137255 0.        ]
  ...
  [0.48235294 0.34509804 0.21568627]
  [0.46666667 0.3254902  0.19607843]
  [0.47843137 0.34117647 0.22352941]]

 [[0.09803922 0.09411765 0.08235294]
  [0.0627451  0.02745098 0.        ]
  [0.19215686 0.10588235 0.03137255]
  ...
  [0.4627451  0.32941176 0.19607843]
  [0.47058824 0.32941176 0.19607843]
  [0.42745098 0.28627451 0.16470588]]

 ...

 [[0.81568627 0.66666667 0.37647059]
  [0.78823529 0.6        0.13333333]
  [0.77647059 0.63137255 0.10196078]
  ...
  [0.62745098 0.52156863 0.2745098 ]
  [0.21960784 0.12156863 0.02745098]
  [0.20784314 0.13333333 0.07843137]]

 [[0.70588235 0.54509804 0.37647059]
  [0.67843137 0.48235294 0.16470588]


In [4]:
from tensorflow.keras import layers

In [5]:
def resnet_50():
    input_shape = (32, 32, 3)
    num_classes = 10
    
    inputs = tf.keras.Input(shape=input_shape)
    
    x = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(inputs)
    
    num_blocks = [3, 3, 3]
    num_filters = [16, 32, 64]
    
    for block_idx in range(len(num_blocks)):
        for layer_idx in range(num_blocks[block_idx]):
            strides = 1
            if block_idx > 0 and layer_idx == 0:
                strides = 2  
            
            y = layers.BatchNormalization()(x)
            y = layers.Activation('relu')(y)
            y = layers.Conv2D(num_filters[block_idx], (3, 3), strides=strides, padding='same')(y)
            y = layers.BatchNormalization()(y)
            y = layers.Activation('relu')(y)
            y = layers.Conv2D(num_filters[block_idx], (3, 3), padding='same')(y)
            
            if block_idx > 0 and layer_idx == 0:
                x = layers.Conv2D(num_filters[block_idx], (1, 1), strides=strides, padding='same')(x)
            
            x = layers.Add()([x, y])
        
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, x)
    
    return model


In [6]:
model = resnet_50()
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])


In [12]:
model.fit(x_train, y_train, batch_size = 256, epochs=15) # 15 epochs

Epoch 1/15
196/196 [==============================] - 235s 1s/step - loss: 1.3942 - sparse_categorical_accuracy: 0.4864
Epoch 2/15
196/196 [==============================] - 153s 779ms/step - loss: 1.0664 - sparse_categorical_accuracy: 0.6131
Epoch 3/15
196/196 [==============================] - 271s 1s/step - loss: 0.9012 - sparse_categorical_accuracy: 0.6758
Epoch 4/15
196/196 [==============================] - 208s 1s/step - loss: 0.7924 - sparse_categorical_accuracy: 0.7174
Epoch 5/15
196/196 [==============================] - 205s 1s/step - loss: 0.7064 - sparse_categorical_accuracy: 0.7484
Epoch 6/15
196/196 [==============================] - 165s 835ms/step - loss: 0.6339 - sparse_categorical_accuracy: 0.7754
Epoch 7/15
196/196 [==============================] - 162s 827ms/step - loss: 0.5670 - sparse_categorical_accuracy: 0.7992
Epoch 8/15
196/196 [==============================] - 177s 904ms/step - loss: 0.5056 - sparse_categorical_accuracy: 0.8216
Epoch 9/15
196/196 [========

In [39]:
y_pred = model.predict(x_test)
print("Test accuracy:", model.evaluate(x_test, y_test))
print(y_pred[0].argmax().round())
print(y_test[0])

313/313 [==============================] - 38s 123ms/step
Test accuracy: 0.84145644
3
[3]


In [50]:
from sklearn.metrics import f1_score
print("F1 score = ", f1_score(y_test, y_pred.argmax(axis=1), average='macro'))

F1 score = 0.8321040814919576
